# Собираем и чистим датасет для обучения

In [2]:
import pandas as pd
import numpy as np

dir = 'data_init/'

## Books

In [102]:
data1 = pd.read_csv(dir + 'cat_1.csv', sep=';', encoding='windows-1251')
data2 = pd.read_csv(dir + 'cat_2.csv', sep=';', encoding='windows-1251')
data3 = pd.read_csv(dir + 'cat_3.csv', sep=';', encoding='windows-1251')

data = pd.concat([data1, data2, data3]).reset_index(drop=True)
print(data.shape)
data.head()

(2179125, 13)


,recId,aut,title,place,publ,yea,lan,rubrics,person,serial,material,biblevel,ager
0,1,Устинова Татьяна Витальевна,Призрак Канта,Москва,Эксмо,2018,rus,Художественная литература : Российский детектив,NaN,Татьяна Устинова. Первая среди лучших,Текст. материал,Монография,16+
1,2,NaN,Наука и жизнь,Москва,Редакция журнала «Наука и жизнь»,1938-,rus,Наука. Науковедение,NaN,NaN,Текст. материал,Периодика,12+
2,3,NaN,Наука и жизнь,NaN,NaN,2018,rus,Наука. Науковедение,NaN,NaN,Текст. материал,Монография,12+
3,4,NaN,Наука и жизнь,NaN,NaN,2018,rus,Наука. Науковедение,NaN,NaN,Текст. материал,Монография,12+
4,5,NaN,Наука и жизнь,NaN,NaN,2018,rus,Наука. Науковедение,NaN,NaN,Текст. материал,Монография,12+


In [103]:
# дропаем с пустым названием
to_delete = data[data['title'].isna()]
print('to delete:', to_delete.shape[0])
         
books_map = to_delete[['recId']].rename(columns={'recId': 'recId_in'})
books_map['recId_out'] = np.nan
data.drop(index=to_delete.index, inplace=True)
print('data:', data.shape)

to delete: 30673
data: (2148452, 13)


In [104]:
# дропаем ненужные колонки
data.drop(['place', 'publ', 'yea', 'lan', 'material', 'biblevel'], axis=1, inplace=True)

In [105]:
data.head()

,recId,aut,title,rubrics,person,serial,ager
0,1,Устинова Татьяна Витальевна,Призрак Канта,Художественная литература : Российский детектив,NaN,Татьяна Устинова. Первая среди лучших,16+
1,2,NaN,Наука и жизнь,Наука. Науковедение,NaN,NaN,12+
2,3,NaN,Наука и жизнь,Наука. Науковедение,NaN,NaN,12+
3,4,NaN,Наука и жизнь,Наука. Науковедение,NaN,NaN,12+
4,5,NaN,Наука и жизнь,Наука. Науковедение,NaN,NaN,12+


In [106]:
# отделяем уникальные (чистые) данные и дубли

dupl = data[data[['title', 'aut']].duplicated(keep='first')]
data.drop_duplicates(['title', 'aut'], keep='first', inplace=True) 
data.shape, dupl.shape, data.shape[0] + dupl.shape[0]

((1107638, 7), (1040814, 7), 2148452)

In [107]:
# мерджим чистые данные и дубли, чтобы сделать маппинг и дозаполнить чистые данные

full = pd.concat([data, dupl]).reset_index(drop=True)
extra_data = pd.merge(left=data, right=full, how='left', on=['title', 'aut'])

print(extra_data.shape)
extra_data.head()

(2148452, 12)


,recId_x,aut,title,rubrics_x,person_x,serial_x,ager_x,recId_y,rubrics_y,person_y,serial_y,ager_y
0,1,Устинова Татьяна Витальевна,Призрак Канта,Художественная литература : Российский детектив,NaN,Татьяна Устинова. Первая среди лучших,16+,1,Художественная литература : Российский детектив,NaN,Татьяна Устинова. Первая среди лучших,16+
1,1,Устинова Татьяна Витальевна,Призрак Канта,Художественная литература : Российский детектив,NaN,Татьяна Устинова. Первая среди лучших,16+,1656353,Художественная литература : Российский детекти...,NaN,Татьяна Устинова. Первая среди лучших : Русски...,16+
2,1,Устинова Татьяна Витальевна,Призрак Канта,Художественная литература : Российский детектив,NaN,Татьяна Устинова. Первая среди лучших,16+,2501333,Художественная литература,NaN,Первая среди лучших,16+
3,2,NaN,Наука и жизнь,Наука. Науковедение,NaN,NaN,12+,2,Наука. Науковедение,NaN,NaN,12+
4,2,NaN,Наука и жизнь,Наука. Науковедение,NaN,NaN,12+,3,Наука. Науковедение,NaN,NaN,12+


In [108]:
# делаем маппинг

temp = extra_data[['recId_x', 'recId_y']].copy()
temp.rename(columns={'recId_x': 'recId_out', 'recId_y': 'recId_in'}, inplace=True)
temp = temp.reindex(columns=['recId_in', 'recId_out'])

books_map = pd.concat([books_map, temp])
books_map = books_map.astype({"recId_out": "Int64"})
books_map = books_map.reset_index(drop=True)
print(books_map.shape)
books_map.head()

(2179125, 2)


,recId_in,recId_out
0,20,<NA>
1,24,<NA>
2,29,<NA>
3,118,<NA>
4,119,<NA>


In [114]:
# сохраняем маппинг
books_map.to_csv('data/books_map.csv',encoding='utf-8')

In [116]:
# сохраняем books

data.to_csv('data/books.csv',encoding='utf-8')

In [117]:
data.shape

(1107638, 7)

## Interactions

In [3]:
data1 = pd.read_csv(dir + 'circulaton_1.csv', sep=';', encoding='windows-1251')
data2 = pd.read_csv(dir + 'circulaton_2.csv', sep=';', encoding='windows-1251')
data3 = pd.read_csv(dir + 'circulaton_3.csv', sep=';', encoding='windows-1251')
data4 = pd.read_csv(dir + 'circulaton_4.csv', sep=';', encoding='windows-1251')
data5 = pd.read_csv(dir + 'circulaton_5.csv', sep=';', encoding='windows-1251')
data6 = pd.read_csv(dir + 'circulaton_6.csv', sep=';', encoding='windows-1251')
data7 = pd.read_csv(dir + 'circulaton_7.csv', sep=';', encoding='windows-1251')
data8 = pd.read_csv(dir + 'circulaton_8.csv', sep=';', encoding='windows-1251')
data9 = pd.read_csv(dir + 'circulaton_9.csv', sep=';', encoding='windows-1251')
data10 = pd.read_csv(dir + 'circulaton_10.csv', sep=';', encoding='windows-1251')
data11 = pd.read_csv(dir + 'circulaton_11.csv', sep=';', encoding='windows-1251')
data12 = pd.read_csv(dir + 'circulaton_12.csv', sep=';', encoding='windows-1251')
data13 = pd.read_csv(dir + 'circulaton_13.csv', sep=';', encoding='windows-1251')
data14 = pd.read_csv(dir + 'circulaton_14.csv', sep=';', encoding='windows-1251')
data15 = pd.read_csv(dir + 'circulaton_15.csv', sep=';', encoding='windows-1251')
data16 = pd.read_csv(dir + 'circulaton_16.csv', sep=';', encoding='windows-1251')

data = pd.concat([data1, data2, data3, data4, data5, data6, data7, data8, data9, data10, data11, data12, data13, data14, data15, data16]).reset_index(drop=True)

print(data.shape)
data.head()

(15714320, 10)


,circulationID,catalogueRecordID,barcode,startDate,finishDate,readerID,bookpointID,state,Unnamed: 8,Unnamed: 9
0,2950477,12555,381002128634,01.09.2019,01.09.2019,1188,46,Размещен в фонде,NaN,NaN
1,2950479,841695,1086000549854,01.09.2019,01.09.2019,23276,56,Размещен в фонде,NaN,NaN
2,2950480,553535,1085000292739,01.09.2019,15.10.2019,23128,55,Размещен в фонде,NaN,NaN
3,2950481,272495,1085000292722,01.09.2019,15.10.2019,23128,55,Размещен в фонде,NaN,NaN
4,2950482,103580,181001063192,01.09.2019,01.10.2019,44894,99,Переоформлен,NaN,NaN


In [4]:
data.drop(['circulationID', 'barcode', 'finishDate', 'bookpointID', 'state', 'Unnamed: 8', 'Unnamed: 9'], axis=1, inplace=True)
data.rename(columns={'catalogueRecordID': 'recId', 'startDate': 'date', 'readerID': 'user_id'}, inplace=True)

data.drop_duplicates(keep='first', inplace=True)

In [5]:
print(data.shape)
data.head()

(13306480, 3)


,recId,date,user_id
0,12555,01.09.2019,1188
1,841695,01.09.2019,23276
2,553535,01.09.2019,23128
3,272495,01.09.2019,23128
4,103580,01.09.2019,44894


In [6]:
users, items, interactions = data.user_id.nunique(), data.recId.nunique(), data.shape[0]

print('# users: ', users)
print('# items: ', items)
print('# interactions: ', interactions)

# users:  555611
# items:  654978
# interactions:  13306480


#### перекодируем с учетом "чистых" книг

In [7]:
books_map = pd.read_csv('data/books_map.csv').drop('Unnamed: 0', axis=1)
print(books_map.shape)
books_map.head()

(2179125, 2)


,recId_in,recId_out
0,20,NaN
1,24,NaN
2,29,NaN
3,118,NaN
4,119,NaN


In [8]:
books_map.rename(columns={'recId_in': 'recId'}, inplace=True)
books_map.head()

,recId,recId_out
0,20,NaN
1,24,NaN
2,29,NaN
3,118,NaN
4,119,NaN


In [9]:
temp = pd.merge(left=data, right=books_map, how='left', on='recId')

In [10]:
print('нет книг в каталоге, %:', temp['recId_out'].isnull().sum()/ temp.shape[0]*100)

нет книг в каталоге, %: 1.716607246995449


In [11]:
temp.dropna(axis=0, inplace=True)
temp.drop('recId', axis=1, inplace=True)

In [12]:
temp.rename(columns={'recId_out': 'item_id'}, inplace=True)
temp.drop_duplicates(keep='first', inplace=True)


In [13]:
interactions = temp.copy()
interactions = interactions.astype({"item_id": "Int64"})
print(interactions.shape)
interactions.head()

(11383578, 3)


,date,user_id,item_id
0,01.09.2019,1188,12555
1,01.09.2019,23276,252072
2,01.09.2019,23128,237826
4,01.09.2019,44894,103580
5,01.09.2019,44894,244035


In [14]:
users, items, inter = interactions.user_id.nunique(), interactions.item_id.nunique(), interactions.shape[0]

print('# users: ', users)
print('# items: ', items)
print('# interactions: ', inter)

# users:  553767
# items:  355565
# interactions:  11383578


In [15]:
interactions['date'] = pd.to_datetime(interactions['date'], format="%d.%m.%Y")

In [16]:
interactions.head()

,date,user_id,item_id
0,2019-09-01,1188,12555
1,2019-09-01,23276,252072
2,2019-09-01,23128,237826
4,2019-09-01,44894,103580
5,2019-09-01,44894,244035


In [19]:
interactions['timestamp'] = interactions['date']
interactions = interactions.astype({"timestamp": "Int64"})
interactions['timestamp'] = interactions['timestamp'] // 10**9

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  


In [ ]:
# сохраним датасет interactions 

interactions.to_csv('data/interactions.csv', encoding='utf-8', )

In [20]:
interactions.shape

(11383578, 4)